# Segmenting and Clustering Neighborhoods in the city of Toronto 
## Question 2

### Import libraries

In [2]:
import sys
!{sys.executable} -m pip install geocoder

import pandas as pd
import numpy as np
import requests # handle request
from bs4 import BeautifulSoup # use for getting data from URL
import geocoder

print("Libraries imported Success!")

     |████████████████████████████████| 98 kB 501 kB/s eta 0:00:01
Libraries imported Success!


### Load data from URL using BeautifulSoup

In [22]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(URL)

soup = BeautifulSoup(req.content, "html5lib")
table = soup.find('div', {'id':'container'})
# print(soup.prettify()) 

### Prepare dataset 

In [23]:
lst = []
n = 0

all_tds = soup.find_all("td")               # finds all things that starts with <td> and ends with </td>
all_tds_lst = [x.get_text().strip("\n") for x in all_tds]  # convert bs4 list into a list of strings
len_tds = [a for a in range(0, 3)]                      # create list of 0, 1, 2s

for element in all_tds_lst:
    if element == "": 
        break # control for things that goes beyond our last </td>
    if n == 0:
        lst.append([])
        lst[-1].append(element)
        n += 1
    elif n == 2:
        lst[-1].append(element)
        n = 0 # reset n 
    else:
        lst[-1].append(element)
        n += 1

### Now create Data Frame with the list we just made

In [24]:
# create pandas DF
df = pd.DataFrame(data = lst, columns = ['PostalCode', 'Borough', 'Neighborhood'])
df = df[~df.Borough.str.contains("Not assigned")]
df = df[~df.Neighborhood.str.contains("Not assigned")]
df = df.reset_index()
df = df.drop(columns='index', axis=1)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [25]:
df.shape

(103, 3)

## Question 2

In [33]:
df_latlng = pd.read_csv("http://cocl.us/Geospatial_data")
df_latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
# fix Postal Code to PostalCode
df_latlng.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
df_latlng.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
complete_df = pd.merge(df, df_latlng, on = 'PostalCode')
complete_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
